In [ ]:
#importing  all tyhe necessary libraries

from skimage.io import imread #for reading image
import numpy as np # linear algebra
import pandas as pd # pandas for data processing
from tqdm import tqdm_notebook as tqdm
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import matplotlib.pyplot as plt # for plotting the images
import torch #for pytorch tensors
# Neural networks can be constructed using the torch.nn package.
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms

from sklearn.model_selection import train_test_split
# for evaluating the model

from sklearn.metrics import accuracy_score #getting accuracy of a model
from tqdm import tqdm

# PyTorch libraries and modules

import torch
from torch.autograd import Variable
#importing all kinds of activation function
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
#replacing the type names from string to integer
data = pd.read_csv('/kaggle/input/deep-learning-for-msc-coursework-2021/train.csv')
Type = {"Connective" : 0,"Immune" : 1,"Cancer" : 2,"Normal" : 3,}
data.Type = [Type[item] for item in data.Type]
print(data.columns)

In [ ]:
train_img = []

for img_name in tqdm(data['Id']):

    # give the image path

    image_path = '/kaggle/input/deep-learning-for-msc-coursework-2021/train/train/' + str(img_name) + '.png'

    # read images

    img = imread(image_path)
    
    # pixel to float 32

    img = img.astype('float32')

    # normalize the pixel values

    img /= 255.0

    # append the image

    train_img.append(img)



# convert the list to numpy array

train_x = np.array(train_img)

# setting the target

train_y = data['Type'].values

train_x.shape


In [ ]:
# splitting the images to training and validation
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.2)
(train_x.shape, train_y.shape), (val_x.shape, val_y.shape)



print(train_x.shape)
print(train_y.shape)
print(val_x.shape)
print(val_y.shape)

# convert training images into torch tensors
train_x = train_x.reshape(1752, 3, 64, 64) #reshape to cnn format
train_x  = torch.from_numpy(train_x)

val_x = val_x.reshape(438, 3, 64, 64) #reshape to tensor format
val_x  = torch.from_numpy(val_x)

# convert the target into torch tensor
train_y = train_y.astype(int);
train_y = torch.from_numpy(train_y)

val_y = val_y.astype(int);
val_y  = torch.from_numpy(val_y)

# shape of training data
train_x.shape, train_y.shape

In [ ]:
class Net(Module):   #class for the networks
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Define a CNN layer
            Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(16),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            
            # Define a CNN layer
            Conv2d(16, 16, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(16),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            
            
        )
#define linear layer
        self.linear_layers = Sequential(
            Linear(4096, 4)
        )

    # Define forward propagation    
    def forward(self, x):
        x = self.cnn_layers(x)
        
        x = x.view(x.size(0), -1)
        
        x = self.linear_layers(x)
        return x

In [ ]:
# define the model
model = Net()
# define the optimizer
optimizer = Adam(model.parameters(), lr=0.05)
# define the loss function
criterion = CrossEntropyLoss()
# check if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

In [ ]:
def train(epoch):
    model.train()
    tr_loss = 0
    # fetch the training set
    x_train, y_train = Variable(train_x), Variable(train_y)
    # fetch the validation set
    
    x_val, y_val = Variable(val_x), Variable(val_y)
    # convert data into GPU format if gpu is available
    if torch.cuda.is_available():
        x_train = x_train.cuda()
        y_train = y_train.cuda()
        x_val = x_val.cuda()
        y_val = y_val.cuda()

    # clear previous gradient values
    optimizer.zero_grad()
    
    # predict for training and validation set
    output_train = model(x_train)
    output_val = model(x_val)

    # compute training and validation loss
    loss_train = criterion(output_train, y_train)
    loss_val = criterion(output_val, y_val)
    train_losses.append(loss_train)
    val_losses.append(loss_val)

    # compute updated weights for all model parameters
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    if epoch%2 == 0:
        # printing the validation loss
        print('Epoch : ',epoch+1, '\t', 'loss :', loss_val)

In [ ]:
# define the number of epochs
n_epochs = 32
# train_losses to store training losses
train_losses = []
# val_losses to store validation losses
val_losses = []
# train the model for n_epoch number of iterations
for epoch in range(n_epochs):
    train(epoch)

In [ ]:
# predict for training set
with torch.no_grad():
    output = model(train_x)
    
softmax = torch.exp(output).cpu()
prob = list(softmax.numpy())
predictions = np.argmax(prob, axis=1)

# accuracy of training set
accuracy_score(train_y, predictions)

In [ ]:
#predict for validation
with torch.no_grad():
    output = model(val_x
                  )

softmax = torch.exp(output).cpu()
prob = list(softmax.numpy())
predictions = np.argmax(prob, axis=1)

# accuracy for the validation set
accuracy_score(val_y, predictions)

In [ ]:
list1=[]
list2=[]
for i in train_losses:
    list1.append(i.detach().numpy())
   
for  j in val_losses:
    list2.append(i.detach().numpy())
   
plt.plot(list1,label='training loss')
plt.plot(list2,label='validation loss')
plt.legend()
plt.show()


In [ ]:
#to get the final_data which will be used to get the test images
import glob
namelist=[]
final_data=pd.DataFrame(columns=['Pred'])
testdata = glob.glob("/kaggle/input/deep-learning-for-msc-coursework-2021/test/test/*.png")
for filename in testdata:
    #print(filename)
    name = filename.replace("kaggle/input/deep-learning-for-msc-coursework-2021/test/test/","").replace('.png','')
    name=name.replace("/","")
    name=name.strip()
    ##print(name)
    namelist.append(int(name))
#print(namelist)
final_data["Id"]=namelist
final_data.drop(columns=['Pred'],inplace=True)
final_data.head()

In [ ]:
#To get the test_x
test_img = []

for img_name in final_data['Id']:

    # defime the image path

    image_path = '/kaggle/input/deep-learning-for-msc-coursework-2021/test/test/' + str(img_name) + '.png'

    # read the image

    img = imread(image_path)
    
    # convert the type of pixel to float 32

    img = img.astype('float32')

    # normalise the pixel values

    img /= 255.0

    # append the image into the list

    test_img.append(img)



# convert the list to numpy array

test_x = np.array(test_img)


test_x.shape


In [ ]:
# convert training images into torch format
test_x = test_x.reshape(400, 3, 64, 64)
test_x  = torch.from_numpy(test_x)
test_x.shape

In [ ]:
# generate predictions for test set
with torch.no_grad():
    output = model(test_x)

softmax = torch.exp(output).cpu()
prob = list(softmax.numpy())

predictions = np.argmax(prob, axis=1)

final_data['Type']=predictions
final_data.head()

In [ ]:
# convert the integers labels back to the string type
Type = {0 : "Connective",1 : "Immune",2 : "Cancer",3 :"Normal"}
final_data.Type = [Type[item] for item in final_data.Type]
print(final_data)

In [ ]:
# Getting the output to a csv file
new_final_data = final_data.sort_values(by = 'Id')
print(new_final_data.head())

In [ ]:
new_final_data.to_csv("test_26.csv", index = False)